In [1]:
### this will follow the example at https://python.langchain.com/docs/expression_language/cookbook/sql_db

In [2]:
%pip install langchain

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/42/4e/86204994aeb2e4ac367a7fade896b13532eae2430299052eb2c80ca35d2c/langchain-0.0.329-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp<4.0.0,>=3.8.3 from https://files.pythonhosted.org/packages/91/1b/9b5da3748ddd924288d9a946a0f48a34b179330d342c6cdeb89683bcf971/aiohttp-3.8.6-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached aiohttp-3.8.6-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Obtaining dependency information for anyio<4.0 from https://files.pythonhosted.org/packages/19/24/44299477fe7dcc9cb58d0a57d5a7588d6af2ff403fdd2d47a246c91a3246/anyio-3.7.1-py3-none-any.whl.metadata
  Using cached anyio-3.7.1-py3-none-any.whl.metadata (4.7 kB)
  Obtaining dependency information for async-timeout<5.0.0,>=4.0.0 from https://files.pythonhosted.org/packages/a7/fa/e01228c2938de91d47b307831c62ab9e4001e747789d0b05baf779a6488

### setup DB on the studio kernel instance

In [15]:
## setup DB
%pwd

'/root/aws-gen-ai/genai-data-sql'

In [16]:
%wget https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql

UsageError: Line magic function `%wget` not found.


In [10]:
from langchain.prompts import ChatPromptTemplate

template = """Human: Based on the table schema below, write a SQL query and just the SQL, nothing else, that would answer the user's question.:
{schema}


Question: {question}
SQL Query:
"""
prompt = ChatPromptTemplate.from_template(template)

### Athena Connection

In [3]:
from langchain.utilities import SQLDatabase

In [3]:
%pip install "sqlalchemy<2"

  Obtaining dependency information for sqlalchemy<2 from https://files.pythonhosted.org/packages/ce/4b/b5afbaf063b18bd6decffe8d64184ca5ecb25cc6be2ffc1cd8664ac3a5c2/SQLAlchemy-1.4.50-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.2 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install langchain_experimental

  Obtaining dependency information for langchain_experimental from https://files.pythonhosted.org/packages/69/71/c7ace3f7b6580c7cad29549cc5d4a5ee7bb14d5c373b8005862c844b92f4/langchain_experimental-0.0.37-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 1.8 MB/s eta 0:00:00:00:01
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install sqlalchemy-access

  Obtaining dependency information for sqlalchemy-access from https://files.pythonhosted.org/packages/ab/23/ab5a62f9ff084dedfbe4d9b737278c9eb59f158f36e7d26df24aca2ec452/sqlalchemy_access-2.0.2-py3-none-any.whl.metadata
  Obtaining dependency information for SQLAlchemy>=2.0.0 from https://files.pythonhosted.org/packages/a4/90/9bc93131211b628e4339700713647aeecbc194e66ac6de869711c631b0cd/SQLAlchemy-2.0.23-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pyodbc>=4.0.27 from https://files.pythonhosted.org/packages/cd/88/d61d005ccc7d96e5fec99d1faeba16e08368b836550ab54f64d823359119/pyodbc-5.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
INFO: pip is looking at multiple versions of sqlalchemy-access to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of sqlalchemy-access to determine which version is compatible with other requirem

In [8]:
%pip install PyAthena

  Obtaining dependency information for PyAthena from https://files.pythonhosted.org/packages/84/36/6777e899043845ec923dc3b946587ad6599199ab5217a422b4484b4a1080/pyathena-3.0.10-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 839.8 kB/s eta 0:00:000:01
  Obtaining dependency information for urllib3<1.27,>=1.25.4 from https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a890ecf13314e3e149c05270cc644581f77f17fd/urllib3-1.26.18-py2.py3-none-any.whl.metadata
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 758.1 kB/s eta 0:00:0000:01
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.6
    Uninstalling urllib3-2.0.6:
      Successfully uninstalled urllib3-2.0.6

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you ma

In [2]:
import boto3
from botocore.config import Config
from langchain import PromptTemplate,SagemakerEndpoint,SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.chains import create_sql_query_chain
from sqlalchemy import create_engine

"""
Here we will build the required parameter to connect athena and query database.
1. Data is stored in S3 and metadata in Glue metastore.
2. Create a profille which will have access to the required service.
3. if the database exists and s3 buckets exists use them else create.

"""
region = 'us-east-1'
athena_url = f"athena.{region}.amazonaws.com" 
athena_port = '443' #Update, if port is different
athena_db = 'demo-emp-deb-2' #from user defined params
glue_databucket_name='athena-query-bucket-bharsrid'
s3stagingathena = f's3://{glue_databucket_name}/athenaresults/' 
athena_wkgrp = 'primary' 
athena_connection_string = f"awsathena+rest://@{athena_url}:{athena_port}/{athena_db}?s3_staging_dir={s3stagingathena}/&work_group={athena_wkgrp}"

"""
Under the hood, LangChain uses SQLAlchemy to connect to SQL databases. 
The SQLDatabaseChain can therefore be used with any SQL dialect 
supported by SQLAlchemy, such as MS SQL, MySQL, MariaDB, PostgreSQL, 
Oracle SQL, and SQLite. 
"""
print(athena_connection_string)
athena_engine = create_engine(athena_connection_string, echo=True)
athena_db_connection = SQLDatabase(athena_engine)


awsathena+rest://@athena.us-east-1.amazonaws.com:443/demo-emp-deb-2?s3_staging_dir=s3://athena-query-bucket-bharsrid/athenaresults//&work_group=primary


In [6]:
def get_schema(_):
    return athena_db_connection.get_table_info()

In [7]:
def run_query(query):
    return athena_db_connection.run(query)

In [8]:
inference_modifier = {
    "temperature": 1,
    "top_p": .999,
    "top_k": 250,
    "max_tokens_to_sample": 300,
    "stop_sequences": ["\n\nSQL Query:"]
}

In [11]:
from langchain.chat_models import BedrockChat
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

chat = BedrockChat(model_id="anthropic.claude-v2", model_kwargs=inference_modifier)

# model = ChatOpenAI()

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | chat.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [12]:
sql_response.invoke({"question": "How many employees are there?"})



2023-11-02 18:07:28,103 INFO sqlalchemy.engine.Engine SELECT details.employee_id, details."first name", details."last name" 
FROM details LIMIT %(param_1)s
2023-11-02 18:07:28,106 INFO sqlalchemy.engine.Engine [generated in 0.00235s] {'param_1': 3}
2023-11-02 18:07:29,578 INFO sqlalchemy.engine.Engine SELECT location.employee_id, location.location 
FROM location LIMIT %(param_1)s
2023-11-02 18:07:29,579 INFO sqlalchemy.engine.Engine [generated in 0.00105s] {'param_1': 3}


/opt/conda/lib/python3.8/site-packages/langchain/llms/bedrock.py:50: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: 

Human: Based on the table schema below, write a SQL query and just the SQL, nothing else, that would answer the user's question.:

CREATE EXTERNAL TABLE details (
	employee_id INT,
	`first name` STRING,
	`last name` STRING
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
	'field.delim' = ','
)
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://employee-db-genai-demo/employee/Details/'
TBLPROPERTIES (
	'CrawlerSchemaDeserializerVersion' = '1.0',
	'CrawlerSchemaSerializerVersion' = '1.0',
	'UPDATED_BY_CRAWLER' = 'employee-db-demo-crawler',
	'areColumnsQuoted' = 'false',
	'averageRecordSize' = '18',
	'classification' = 'csv',
	'columnsOrdered' = 'true',
	'compressionType' = '

' SELECT COUNT(DISTINCT employee_id) \nFROM details'

In [13]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [14]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | chat
)

In [15]:
full_chain.invoke({"question": "How many employees are there?"})

2023-11-02 18:07:59,606 INFO sqlalchemy.engine.Engine SELECT details.employee_id, details."first name", details."last name" 
FROM details LIMIT %(param_1)s
2023-11-02 18:07:59,607 INFO sqlalchemy.engine.Engine [cached since 31.5s ago] {'param_1': 3}
2023-11-02 18:08:01,901 INFO sqlalchemy.engine.Engine SELECT location.employee_id, location.location 
FROM location LIMIT %(param_1)s
2023-11-02 18:08:01,902 INFO sqlalchemy.engine.Engine [cached since 32.32s ago] {'param_1': 3}


/opt/conda/lib/python3.8/site-packages/langchain/llms/bedrock.py:50: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: 

Human: Based on the table schema below, write a SQL query and just the SQL, nothing else, that would answer the user's question.:

CREATE EXTERNAL TABLE details (
	employee_id INT,
	`first name` STRING,
	`last name` STRING
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
	'field.delim' = ','
)
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://employee-db-genai-demo/employee/Details/'
TBLPROPERTIES (
	'CrawlerSchemaDeserializerVersion' = '1.0',
	'CrawlerSchemaSerializerVersion' = '1.0',
	'UPDATED_BY_CRAWLER' = 'employee-db-demo-crawler',
	'areColumnsQuoted' = 'false',
	'averageRecordSize' = '18',
	'classification' = 'csv',
	'columnsOrdered' = 'true',
	'compressionType' = '

2023-11-02 18:08:04,754 INFO sqlalchemy.engine.Engine SELECT details.employee_id, details."first name", details."last name" 
FROM details LIMIT %(param_1)s
2023-11-02 18:08:04,756 INFO sqlalchemy.engine.Engine [cached since 36.65s ago] {'param_1': 3}
2023-11-02 18:08:06,006 INFO sqlalchemy.engine.Engine SELECT location.employee_id, location.location 
FROM location LIMIT %(param_1)s
2023-11-02 18:08:06,009 INFO sqlalchemy.engine.Engine [cached since 36.43s ago] {'param_1': 3}


NameError: name 'db' is not defined